# [Learning to learn by gradient descent](https://arxiv.org/abs/1606.04474)

- SEOHASONG - Korea Univ.
- 2018.03.25
- Deep Paper Study 신촌


## Abstract

경영, 공학, 정치 등 다양한 분야에서 사람들은 목적달성을 위해 결과에 영향을 주는 변수를 찾습니다.

변수를 찾는 과정은 사람이 직접 찾아낼 수 도 있지만,
기계학습이 발전하면서 컴퓨터가 자동으로 변수를 찾는 방식이 주목받고 있습니다.

보통 기계학습을 위한 알고리즘은 예측이 틀린 정도가 최소가 되도록 하는 최적화 알고리즘을 의미합니다.
기계학습은 이 최적화 알고리즘과 데이터를 통해 자동으로 변수를 찾도록 설계됩니다.

여기서 컴퓨터가 사용할 알고리즘은 사람이 직접 만들어 줘야 하며,
이는 완전한 자동화가 아니라고 볼 수도 있습니다.

이 논문에서는 사람이 직접 만들던 최적화 알고리즘을 학습하는 문제를 논의합니다.
기존에는 사람이 만든 최적화 알고리즘을 통해 변수를 학습하였다면,
더 나아가 최적화 알고리즘을 변수로 두고 그것이 무엇일지 자동으로 학습하는 문제를 다룹니다.

이 논문에서 제시한 알고리즘은 LSTM에 적용되었으며, 일반적인 사람이 만든 LSTM보다
학습 데이터를 더 잘 학습하고, 새로운 예측에 대해서도 더 잘 예측하였습니다.